In [5]:
"""
classificacao_genero_final.py
Classifica usando IBGE + Base Brasileira + Heurística
Gera 2 arquivos: dataset classificado + dicionário de nomes
"""

import pandas as pd
import re

print("="*60)
print("CLASSIFICAÇÃO FINAL - IBGE + BASE BRASILEIRA")
print("="*60 + "\n")

# ==================== 1. BASE IBGE MANUAL ====================
print("1. Carregando base IBGE manual...")

nomes_ibge_m = {
    'miguel', 'arthur', 'heitor', 'bernardo', 'theo', 'davi', 'gabriel', 'pedro', 
    'lucas', 'matheus', 'rafael', 'samuel', 'gustavo', 'felipe', 'bruno', 'leonardo',
    'guilherme', 'fernando', 'carlos', 'ricardo', 'roberto', 'paulo', 'jose',
    'joao', 'antonio', 'marcos', 'marcelo', 'andre', 'luis', 'rodrigo', 'daniel',
    'diego', 'fabio', 'caio', 'vitor', 'henrique', 'mauricio', 'sergio', 'thiago',
    'wellington', 'william', 'renato', 'eduardo', 'alexandre', 'cesar', 'nelson',
    'jorge', 'mario', 'alberto', 'julio', 'francisco', 'manuel', 'oseias',
    'caique', 'zacarias', 'manoel', 'matias', 'ademir', 'adilson', 'adriano',
    'ademario', 'adalberto', 'afonso', 'abilio', 'abraao', 'acelino'
}

nomes_ibge_f = {
    'maria', 'ana', 'julia', 'laura', 'beatriz', 'alice', 'sophia', 'helena',
    'isabela', 'manuela', 'giovanna', 'mariana', 'valentina', 'clara', 'carolina',
    'fernanda', 'amanda', 'jessica', 'juliana', 'patricia', 'aline', 'camila',
    'luciana', 'priscila', 'adriana', 'cristina', 'daniela', 'gabriela', 'leticia',
    'renata', 'roberta', 'silvia', 'tatiana', 'vanessa', 'monica', 'sandra',
    'rosa', 'barbara', 'elaine', 'grace', 'kelly', 'michele', 'natalia', 'paula',
    'raquel', 'simone', 'teresa', 'vera', 'angelica', 'bianca', 'carla', 'debora',
    'elizabete', 'fabiana', 'glaucia', 'ines', 'janaina', 'karine',
    'larissa', 'lilian', 'evelin', 'shirlei', 'eliana', 'daci', 'neziani',
    'stephanie', 'deise', 'hellen', 'rayane', 'adrielle', 'adiene', 'adelina'
}

print(f"IBGE: {len(nomes_ibge_m)} M, {len(nomes_ibge_f)} F\n")

# ==================== 2. CARREGAR BASE BRASILEIRA ====================
print("2. Carregando base brasileira de nomes...")

#base dados do portal público: https://brasil.io/dataset/genero-nomes/files/
df_base = pd.read_csv('nomes.csv')
print(f"Base carregada: {len(df_base)} nomes\n")

# ==================== 3. FUNÇÕES ====================

def normalizar_nome(nome):
    """Normaliza nome: minúscula e sem acentos"""
    if pd.isna(nome):
        return None
    
    nome = str(nome).lower().strip()
    nome = re.sub(r'[áàâãä]', 'a', nome)
    nome = re.sub(r'[éèêë]', 'e', nome)
    nome = re.sub(r'[íìîï]', 'i', nome)
    nome = re.sub(r'[óòôõö]', 'o', nome)
    nome = re.sub(r'[úùûü]', 'u', nome)
    nome = re.sub(r'[ç]', 'c', nome)
    
    return nome

def extrair_primeiro_nome(nome_completo):
    """Extrai primeiro nome e normaliza"""
    if pd.isna(nome_completo) or not nome_completo:
        return None
    
    nome = str(nome_completo).strip()
    primeiro = nome.split()[0] if nome else None
    
    return normalizar_nome(primeiro) if primeiro else None

def classificar_heuristica(nome):
    """Heurística de terminação"""
    if not nome:
        return None, 0.0
    
    ultima = nome[-1].lower()
    
    if ultima == 'a':
        return 'F', 0.6
    elif ultima == 'o':
        return 'M', 0.6
    elif ultima == 's' and len(nome) > 2 and nome[-2:] == 'as':
        return 'M', 0.5
    
    return None, 0.2

# ==================== 4. CRIAR DICIONÁRIO COMPLETO ====================
print("3. Criando dicionário completo de nomes...\n")

dicionario = {}

# 4.1 Adicionar nomes IBGE (maior prioridade)
print("   a) Adicionando nomes IBGE...")
for nome in nomes_ibge_m:
    nome_norm = normalizar_nome(nome)
    if nome_norm:
        dicionario[nome_norm] = {
            'nome': nome_norm,
            'genero': 'M',
            'confianca': 0.95,
            'fonte': 'ibge_manual'
        }

for nome in nomes_ibge_f:
    nome_norm = normalizar_nome(nome)
    if nome_norm:
        dicionario[nome_norm] = {
            'nome': nome_norm,
            'genero': 'F',
            'confianca': 0.95,
            'fonte': 'ibge_manual'
        }

print(f"{len(dicionario)} nomes IBGE")

# 4.2 Adicionar base brasileira (não sobrescrever IBGE)
print("   b) Adicionando base brasileira...")
contador_base = 0

for idx, row in df_base.iterrows():
    nome_norm = normalizar_nome(row['first_name'])
    
    if nome_norm and nome_norm not in dicionario:  # Não sobrescrever IBGE
        genero = row['classification']
        ratio = row['ratio']
        
        # Confiança baseada no ratio
        if ratio >= 0.95:
            conf = 0.90
        elif ratio >= 0.85:
            conf = 0.80
        else:
            conf = 0.70
        
        dicionario[nome_norm] = {
            'nome': nome_norm,
            'genero': genero,
            'confianca': conf,
            'fonte': 'base_brasileira'
        }
        contador_base += 1
    
    if (idx + 1) % 20000 == 0:
        print(f"      Processados: {idx+1}/{len(df_base)}")

print(f"+{contador_base} nomes da base brasileira")
print(f"\nDicionário total: {len(dicionario)} nomes\n")

# ==================== 5. CARREGAR DATASET ====================
print("4. Carregando dataset TJAC...")

df = pd.read_csv('processos_tjac_completo.csv')
print(f"{len(df)} processos\n")

# ==================== 6. CLASSIFICAR DATASET ====================
print("5. Classificando todas as colunas...\n")

colunas = ['juiz', 'requerente', 'advogado_requerente', 'advogado_requerido']

for coluna in colunas:
    print(f"   Processando: {coluna}...")
    
    generos = []
    
    for nome_completo in df[coluna]:
        primeiro = extrair_primeiro_nome(nome_completo)
        
        if primeiro and primeiro in dicionario:
            # Encontrado no dicionário
            generos.append(dicionario[primeiro]['genero'])
        elif primeiro:
            # Fallback: heurística
            gen_heur, conf_heur = classificar_heuristica(primeiro)
            if gen_heur and conf_heur >= 0.5:
                generos.append(gen_heur)
                # Adicionar ao dicionário
                if primeiro not in dicionario:
                    dicionario[primeiro] = {
                        'nome': primeiro,
                        'genero': gen_heur,
                        'confianca': conf_heur,
                        'fonte': 'heuristica'
                    }
            else:
                generos.append(None)
        else:
            generos.append(None)
    
    # Adicionar coluna ao dataset
    df[f'genero_{coluna}'] = generos
    
    # Estatísticas
    total = len(df)
    masc = df[f'genero_{coluna}'].eq('M').sum()
    fem = df[f'genero_{coluna}'].eq('F').sum()
    nulo = df[f'genero_{coluna}'].isna().sum()
    
    print(f"      M: {masc} ({masc/total*100:.1f}%)")
    print(f"      F: {fem} ({fem/total*100:.1f}%)")
    print(f"      Nulo: {nulo} ({nulo/total*100:.1f}%)")
    print()

# ==================== 7. SALVAR ARQUIVOS ====================
print("6. Salvando arquivos...\n")

# 7.1 Dataset com classificações (SÓ GÊNERO)
df.to_csv('processos_tjac_com_genero.csv', index=False, encoding='utf-8-sig')
print(f"Dataset: processos_tjac_com_genero.csv")

# 7.2 Dicionário de nomes (nome, gênero, confiança, fonte)
df_dict = pd.DataFrame(list(dicionario.values()))
df_dict = df_dict.sort_values('nome')
df_dict.to_csv('dicionario_nomes_classificacao.csv', index=False, encoding='utf-8-sig')
print(f"Dicionário: dicionario_nomes_classificacao.csv")

# ==================== 8. RESUMO FINAL ====================
print("\n" + "="*60)
print("RESUMO FINAL")
print("="*60)
print(f"\nDataset: {len(df)} processos classificados")
print(f"Dicionário: {len(df_dict)} nomes únicos")

print(f"\nFontes do dicionário:")
print(df_dict['fonte'].value_counts())

print(f"\nDistribuição de gêneros no dicionário:")
print(df_dict['genero'].value_counts())

print(f"\nConfiança média: {df_dict['confianca'].mean():.2f}")

print("\n" + "="*60)
print("ANÁLISE - JUÍZES")
print("="*60)

df_juizes = df[df['genero_juiz'].notna()]
total_juizes = len(df)
ident = len(df_juizes)

print(f"\nIdentificados: {ident}/{total_juizes} ({ident/total_juizes*100:.1f}%)")
print(f"  Homens: {(df['genero_juiz'] == 'M').sum()}")
print(f"  Mulheres: {(df['genero_juiz'] == 'F').sum()}")

print("\n" + "="*60)
print("CLASSIFICAÇÃO CONCLUÍDA!")
print("="*60)

CLASSIFICAÇÃO FINAL - IBGE + BASE BRASILEIRA

1. Carregando base IBGE manual...
IBGE: 66 M, 72 F

2. Carregando base brasileira de nomes...
Base carregada: 100787 nomes

3. Criando dicionário completo de nomes...

   a) Adicionando nomes IBGE...
138 nomes IBGE
   b) Adicionando base brasileira...
      Processados: 20000/100787
      Processados: 40000/100787
      Processados: 60000/100787
      Processados: 80000/100787
      Processados: 100000/100787
+100650 nomes da base brasileira

Dicionário total: 100788 nomes

4. Carregando dataset TJAC...
1200 processos

5. Classificando todas as colunas...

   Processando: juiz...
      M: 439 (36.6%)
      F: 760 (63.3%)
      Nulo: 1 (0.1%)

   Processando: requerente...
      M: 548 (45.7%)
      F: 589 (49.1%)
      Nulo: 63 (5.2%)

   Processando: advogado_requerente...
      M: 671 (55.9%)
      F: 245 (20.4%)
      Nulo: 284 (23.7%)

   Processando: advogado_requerido...
      M: 705 (58.8%)
      F: 154 (12.8%)
      Nulo: 341 (28.4%

In [4]:
"""
calcular_confianca_real.py
Calcula confiança baseada nos nomes EFETIVAMENTE usados no dataset
"""

import pandas as pd
import re

print("="*60)
print("CÁLCULO CORRETO DA CONFIANÇA")
print("="*60 + "\n")

# Carregar arquivos
df = pd.read_csv('processos_tjac_com_genero.csv')
df_dict = pd.read_csv('dicionario_nomes_classificacao.csv')

print("1. Arquivos carregados\n")

# Função para extrair primeiro nome
def normalizar_nome(nome):
    if pd.isna(nome):
        return None
    nome = str(nome).lower().strip()
    nome = re.sub(r'[áàâãä]', 'a', nome)
    nome = re.sub(r'[éèêë]', 'e', nome)
    nome = re.sub(r'[íìîï]', 'i', nome)
    nome = re.sub(r'[óòôõö]', 'o', nome)
    nome = re.sub(r'[úùûü]', 'u', nome)
    nome = re.sub(r'[ç]', 'c', nome)
    return nome

def extrair_primeiro_nome(nome_completo):
    if pd.isna(nome_completo):
        return None
    nome = str(nome_completo).strip()
    primeiro = nome.split()[0] if nome else None
    return normalizar_nome(primeiro) if primeiro else None

# Coletar TODOS os nomes usados no dataset (com frequência)
print("2. Coletando nomes usados no dataset...\n")

colunas = ['juiz', 'requerente', 'advogado_requerente', 'advogado_requerido']

# Dicionário: nome → frequência de uso
nomes_usados = {}

for coluna in colunas:
    for nome_completo in df[coluna].dropna():
        primeiro = extrair_primeiro_nome(nome_completo)
        if primeiro:
            nomes_usados[primeiro] = nomes_usados.get(primeiro, 0) + 1

print(f"Total de nomes únicos usados: {len(nomes_usados)}")
print(f"Total de ocorrências: {sum(nomes_usados.values())}\n")

# Buscar confiança de cada nome usado
print("3. Calculando confiança média ponderada...\n")

# Criar dicionário para lookup rápido
dict_lookup = df_dict.set_index('nome')[['confianca', 'fonte']].to_dict('index')

soma_ponderada = 0
total_ocorrencias = 0
nomes_por_fonte = {}

for nome, freq in nomes_usados.items():
    if nome in dict_lookup:
        conf = dict_lookup[nome]['confianca']
        fonte = dict_lookup[nome]['fonte']
        
        soma_ponderada += conf * freq
        total_ocorrencias += freq
        
        # Contar por fonte
        if fonte not in nomes_por_fonte:
            nomes_por_fonte[fonte] = {'qtd_nomes': 0, 'qtd_usos': 0, 'confianca_soma': 0}
        
        nomes_por_fonte[fonte]['qtd_nomes'] += 1
        nomes_por_fonte[fonte]['qtd_usos'] += freq
        nomes_por_fonte[fonte]['confianca_soma'] += conf * freq

# Calcular média ponderada
confianca_real = soma_ponderada / total_ocorrencias if total_ocorrencias > 0 else 0

print("="*60)
print("RESULTADO - CONFIANÇA REAL")
print("="*60)
print(f"\nConfiança média ponderada: {confianca_real:.3f} ({confianca_real*100:.1f}%)")
print(f"  (baseada em {total_ocorrencias} usos de {len(nomes_usados)} nomes únicos)\n")

print("Detalhamento por fonte:")
print("-" * 60)

for fonte in sorted(nomes_por_fonte.keys()):
    info = nomes_por_fonte[fonte]
    qtd_nomes = info['qtd_nomes']
    qtd_usos = info['qtd_usos']
    conf_media = info['confianca_soma'] / qtd_usos if qtd_usos > 0 else 0
    pct_usos = (qtd_usos / total_ocorrencias) * 100
    
    print(f"\n{fonte}:")
    print(f"  Nomes únicos: {qtd_nomes}")
    print(f"  Usos no dataset: {qtd_usos} ({pct_usos:.1f}%)")
    print(f"  Confiança média: {conf_media:.3f}")

print("\n" + "="*60)
print("DETALHAMENTO POR COLUNA")
print("="*60)

for coluna in colunas:
    nomes_col = {}
    
    for nome_completo in df[coluna].dropna():
        primeiro = extrair_primeiro_nome(nome_completo)
        if primeiro and primeiro in dict_lookup:
            nomes_col[primeiro] = nomes_col.get(primeiro, 0) + 1
    
    if nomes_col:
        soma_col = sum(dict_lookup[n]['confianca'] * freq for n, freq in nomes_col.items())
        total_col = sum(nomes_col.values())
        conf_col = soma_col / total_col
        
        print(f"\n{coluna}:")
        print(f"  Confiança média: {conf_col:.3f} ({conf_col*100:.1f}%)")
        print(f"  Usos: {total_col}")

print("\n" + "="*60)

CÁLCULO CORRETO DA CONFIANÇA

1. Arquivos carregados

2. Coletando nomes usados no dataset...

Total de nomes únicos usados: 979
Total de ocorrências: 4223

3. Calculando confiança média ponderada...

RESULTADO - CONFIANÇA REAL

Confiança média ponderada: 0.898 (89.8%)
  (baseada em 4111 usos de 979 nomes únicos)

Detalhamento por fonte:
------------------------------------------------------------

base_brasileira:
  Nomes únicos: 754
  Usos no dataset: 1578 (38.4%)
  Confiança média: 0.895

heuristica:
  Nomes únicos: 34
  Usos no dataset: 365 (8.9%)
  Confiança média: 0.600

ibge_manual:
  Nomes únicos: 107
  Usos no dataset: 2168 (52.7%)
  Confiança média: 0.950

DETALHAMENTO POR COLUNA

juiz:
  Confiança média: 0.900 (90.0%)
  Usos: 1199

requerente:
  Confiança média: 0.910 (91.0%)
  Usos: 1137

advogado_requerente:
  Confiança média: 0.902 (90.2%)
  Usos: 916

advogado_requerido:
  Confiança média: 0.875 (87.5%)
  Usos: 859

